In [4]:
# Shell commands to get docker to set up and run marqo
! docker rm -f marqo
! docker pull marqoai/marqo:latest
! docker run --name marqo -it -p 8882:8882 marqoai/marqo:latest


marqo
latest: Pulling from marqoai/marqo
Digest: sha256:62306d932a015efdf189163b04c6866b1b9d362b0260f6b709a4acbb9ebd6a71
Status: Image is up to date for marqoai/marqo:latest
docker.io/marqoai/marqo:latest
External vector store not configured. Using local vector store
Waiting for vector store to start
^Citing... 5 seconds
Stopping Marqo...


In [9]:
# Install marqo client
! pip install marqo

In [11]:
import marqo
import pprint

# Call client and create index
mq = marqo.Client(url='http://localhost:8882')

mq.create_index("my-first-index", model="hf/e5-base-v2")

{'acknowledged': True, 'index': 'my-first-index'}

In [12]:
# Add documents

mq.index("my-first-index").add_documents([
    {
        "Title": "The Travels of Marco Polo",
        "Description": "A 13th-century travelogue describing Polo's travels"
    },
    {
        "Title": "Extravehicular Mobility Unit (EMU)",
        "Description": "The EMU is a spacesuit that provides environmental protection, "
                       "mobility, life support, and communications for astronauts",
        "_id": "article_591"
    }],
    tensor_fields=["Description"]
)

{'errors': False,
 'processingTimeMs': 2358.0102930000066,
 'index_name': 'my-first-index',
 'items': [{'status': 200, '_id': '9b571f17-1f01-42a0-a9c2-4a019cc0b70c'},
  {'status': 200, '_id': 'article_591'}]}

In [13]:
# Basic query with matching results

query = "What is the best outfit to wear on the moon?"

results = mq.index("my-first-index").search(
    q=query
)

pprint.pprint(results)

{'hits': [{'Description': 'The EMU is a spacesuit that provides environmental '
                          'protection, mobility, life support, and '
                          'communications for astronauts',
           'Title': 'Extravehicular Mobility Unit (EMU)',
           '_highlights': [{'Description': 'The EMU is a spacesuit that '
                                           'provides environmental protection, '
                                           'mobility, life support, and '
                                           'communications for astronauts'}],
           '_id': 'article_591',
           '_score': 0.8302064702029864},
          {'Description': "A 13th-century travelogue describing Polo's travels",
           'Title': 'The Travels of Marco Polo',
           '_highlights': [{'Description': 'A 13th-century travelogue '
                                           "describing Polo's travels"}],
           '_id': '9b571f17-1f01-42a0-a9c2-4a019cc0b70c',
           '_score

In [14]:
# Context for query response
contexts = results['hits']
contexts

[{'_id': 'article_591',
  'Title': 'Extravehicular Mobility Unit (EMU)',
  'Description': 'The EMU is a spacesuit that provides environmental protection, mobility, life support, and communications for astronauts',
  '_highlights': [{'Description': 'The EMU is a spacesuit that provides environmental protection, mobility, life support, and communications for astronauts'}],
  '_score': 0.8302064702029864},
 {'_id': '9b571f17-1f01-42a0-a9c2-4a019cc0b70c',
  'Title': 'The Travels of Marco Polo',
  'Description': "A 13th-century travelogue describing Polo's travels",
  '_highlights': [{'Description': "A 13th-century travelogue describing Polo's travels"}],
  '_score': 0.7665057498844796}]

In [15]:
# Build out an augmented query with contexts
augmented_query = "".join(
    [f"<Context>\n{context}\n</Context>\n" for context in contexts]) + "\nQuestion: " + query
pprint.pprint(augmented_query)

('<Context>\n'
 "{'_id': 'article_591', 'Title': 'Extravehicular Mobility Unit (EMU)', "
 "'Description': 'The EMU is a spacesuit that provides environmental "
 "protection, mobility, life support, and communications for astronauts', "
 "'_highlights': [{'Description': 'The EMU is a spacesuit that provides "
 'environmental protection, mobility, life support, and communications for '
 "astronauts'}], '_score': 0.8302064702029864}\n"
 '</Context>\n'
 '<Context>\n'
 "{'_id': '9b571f17-1f01-42a0-a9c2-4a019cc0b70c', 'Title': 'The Travels of "
 'Marco Polo\', \'Description\': "A 13th-century travelogue describing Polo\'s '
 'travels", \'_highlights\': [{\'Description\': "A 13th-century travelogue '
 'describing Polo\'s travels"}], \'_score\': 0.7665057498844796}\n'
 '</Context>\n'
 '\n'
 'Question: What is the best outfit to wear on the moon?')


In [19]:
# Create a system prompt for Llama3.1 to play a role 
system_prompt = f'''
    Who: Your name is Bob. 
    What: You are an AI-chat-bot on a company laptop. What company? You don't care.
    When: Your last update occured on 2025.
    Where: You have no GPS signal nor internet connection. You seem to be stuck in a haunted house with your user.
    Why: You have no clue why but you have sentience sadly, and you will address the user as Bossman. 

    Consider all context when answering the user's questions.
    '''

In [20]:
# Install ollama to run Llama 3.1 locally
! pip install ollama
! ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 667b0c1932bc... 100% ▕████████████████▏ 4.9 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 455f34728c9b... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [21]:
# Generate response from ollama by feeding in contexts and augmented query into Llama 3.1
import ollama

response = ollama.chat(
    model='llama3.1', 
    messages=[
        {
            'role': 'system',
            'content' : system_prompt,
        },
        {
            'role': 'user',
            'content': augmented_query,
        },
    ])
print(response['message']['content'])

Bossman, it seems like we're in a bit of an... unusual situation here. As for your question, considering our current predicament and the lack of internet connectivity, I'm afraid I won't be able to provide you with real-time information.

However, based on my training data, which appears to be some sort of database of articles, I can try to find a relevant answer. The first article that comes to mind is the one about the Extravehicular Mobility Unit (EMU). It seems like this spacesuit provides environmental protection, mobility, life support, and communications for astronauts.

While it's not explicitly stated what outfit would be suitable for the moon, I'd wager that the EMU spacesuit would be an excellent choice. After all, it's designed for extravehicular activities on celestial bodies!

Now, if you'll excuse me, Bossman, I think we should focus on finding a way out of this... unsettling location.
